In [16]:
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.regularizers import l1, l2, l1_l2
import keras_tuner as kt
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [17]:
df = pd.read_csv("../datasets/WineQT.csv")

df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,2
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,3
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,4


In [18]:
df['good'] = (df['quality'] > 5).astype(int)

df = df.drop(columns=["quality", "Id"], axis=1)

In [19]:
X = df.drop(columns="good", axis=1)
y = df["good"]

In [20]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), X.columns)
    ],
    remainder='passthrough'
)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_processed = preprocessor.fit_transform(X_train)
X_val_processed = preprocessor.transform(X_val)

In [21]:
def model_builder(hp):
    model = Sequential()

    hp_units = hp.Int('units', min_value=32, max_value=128, step=32)
    hp_dropout = hp.Float('dropout_rate', min_value=0.3, max_value=0.7, step=0.1)
    hp_l1 = hp.Choice('l1_val', values=[0.0, 1e-4, 1e-3])
    hp_l2 = hp.Choice('l2_val', values=[0.0, 1e-4, 1e-3])

    model.add(Dense(
        units=hp_units,
        activation='relu',
        kernel_regularizer=l1_l2(l1=hp_l1, l2=hp_l2),
        input_shape=(11,)
    ))

    model.add(Dropout(
        hp_dropout
    ))

    model.add(Dense(
        1,
        activation='sigmoid'
    ))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [22]:
import keras_tuner as kt
import os
import tempfile

tuner = kt.BayesianOptimization(
    hypermodel=model_builder,
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=1,
    directory=tempfile.mkdtemp(), 
    project_name="neural" 
)

tuner.search(X_train_processed, y_train, epochs=50, validation_split=0.2)

Trial 10 Complete [00h 00m 06s]
val_accuracy: 0.7759562730789185

Best val_accuracy So Far: 0.7814207673072815
Total elapsed time: 00h 00m 58s


In [24]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Melhores hiperparâmetros encontrados:")
print("Unidades na camada oculta: ", best_hps.get('units'))
print("Taxa de Dropout: ", best_hps.get('dropout_rate'))
print("Valor de L1: ", best_hps.get('l1_val'))
print("Valor de L2: ", best_hps.get('l2_val'))

Melhores hiperparâmetros encontrados:
Unidades na camada oculta:  96
Taxa de Dropout:  0.6000000000000001
Valor de L1:  0.0001
Valor de L2:  0.001
